In [1]:
# 크롤링 및 추천시스템 위한 라이브러리 호출
import pandas as pd  
import numpy as np   

import requests
from bs4 import BeautifulSoup    
from selenium import webdriver   
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller 

import time    
import re
import math
from tqdm import tqdm_notebook

# 데이터분석 위한 라이브러리 호출

import nltk
from konlpy.tag import Okt

# 그래프
%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz             
from sklearn.tree import export_graphviz

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import matplotlib
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/HMFMMUEX.ttc").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
matplotlib.rcParams['axes.unicode_minus'] = False

from collections import Counter

import warnings
warnings.filterwarnings('ignore')


In [3]:
# 상품페이지 URL 이동을 위한 제품 바코드 확인
df = pd.read_excel('gyobo_best_all.xlsx') # 교보문고 분야별 베스트셀러
df

,순위,바코드,도서명,저자명,출판사명,출간일,정가,카테고리,담당자
0,101,9791190641159,첫 번째 디자인 케이크,지은혜(아이라이크케이크),책밥,20200820,16000,요리,1
1,79,9788982814471,연금술사,파울로 코엘료,문학동네,20181205,12000,소설,1
2,138,9791189727451,기계설비 표준품셈(2022)(CD1장포함)(양장본 HardCover),건설연구원,건설연구원,20220114,34000,기술/공학,1
3,115,9791167900791,그때 그 마음(2022 제67회 현대문학상 수상소설집),정소현 외,현대문학,20211210,15000,소설,1
4,89,9791165343842,대한민국 금기 깨기,김동연,쌤앤파커스,20210728,17000,정치사회,1
...,...,...,...,...,...,...,...,...,...
4596,128,9791136292360,하이큐!! 컴플리트 일러스트레이션 북 끝과 시작,Haruichi Furudate,대원씨아이,20220119,40000,만화,5
4597,114,9788994521626,TOEFL Junior Test RC: Intermediate(Preparation...,CDL ESL Content RD Division,LEARN21,20110901,15500,외국어,5
4598,61,9788920011122,"현대일본정치의이해(2학기, 워크북포함)",강상규,한국방송통신대학교출판문화원,20130725,17600,방송통신대교재,5
4599,19,9791134855475,귀멸의 칼날. 17,고토게 코요하루,학산문화사,20200425,5000,만화,5


In [7]:
df = df[df['담당자']==4] # 담당자 변경
df.reset_index(drop=True, inplace= True) #인덱스 정렬
df

,순위,바코드,도서명,저자명,출판사명,출간일,정가,카테고리,담당자
0,95,9788926159156,최상위 수학 S 초등 3-1(2021),디딤돌 편집부,디딤돌,20201001,15000,초등참고서,4
1,4,9791190826839,오십에 읽는 논어,최종엽,유노북스,20211103,16000,인문,4
2,65,9791161502199,기적의 말하기 영단어 1000(무료 MP3 파일),이시원,시원스쿨닷컴,20190212,9000,외국어,4
3,39,9791191825084,기묘한 미술관,진병관,빅피시,20210908,17800,청소년,4
4,115,9788977991033,건축기술지침 Rev. 2: 건축1,대한건축학회,공간예술사,20170610,38000,기술/공학,4
...,...,...,...,...,...,...,...,...,...
915,140,9788934996361,세계는 왜 싸우는가,김영미,김영사,20190715,14800,정치사회,4
916,51,9791190665629,실무에 바로 쓰는 일잘러의 보고서 작성법,김마라,제이펍,20201117,18000,컴퓨터,4
917,16,9791191043150,나는 나답게 살기로 했다,손힘찬,스튜디오오드리,20210208,13500,인문,4
918,149,9791190670821,2022 렛유인 GSAT 삼성직무적성검사 독학단기완성,정지성,렛유인,20211230,23000,취업/수험서,4


In [8]:
# 크롤링 위한 데이터프레임 만들기
df_keyword = pd.DataFrame( columns=['id', 'klover','comment','keyword','score','total','title'])
df_keyword

,id,klover,comment,keyword,score,total,title


In [9]:
# 크롤링 전처리
def preprocessing(total):
    k_total = int(re.sub('\D','',total))
    return k_total

def keyword_pre(keywords):
    for keyword in keywords:
        keyword_list.append(keyword.text)

def id_pre(ids):
    for id in ids:
        id_list.append(id.text)
        
def klovers_pre(klovers):
    for klover in klovers:
        klover_list.append(klover.text)
        
def comments_pre(comments):
    for comment in comments:
        comment_list.append(comment.text)

In [ ]:
for j in tqdm_notebook(range(41, 120)):
    url = df.iloc[j][1]
    title = df['도서명'][j]

# 크롬 웹 브라우저 실행 (keyword 검색결과)

    chrome_path = chromedriver_autoinstaller.install()
    driver = webdriver.Chrome(chrome_path)
    driver.get('http://www.kyobobook.co.kr/product/detailViewKor.laf?mallGb=KOR&ejkGb=KOR&linkClass=01&barcode={}'.format(url)) # query : 바코드 숫자
    time.sleep(1)
    
    # 회원리뷰 클릭 : 그래야 clova 회원리뷰 목록이 나옴
    try:
        driver.find_element_by_partial_link_text('회원리뷰').click() 
        time.sleep(2)
    
    ## 전처리 함수 완성 ## 

        keyword_list = []
        id_list = []
        klover_list = []
        comment_list = []
    
        total = driver.find_element_by_id('kloverTotal').text
        k_total = preprocessing(total)
        page_n = math.ceil(k_total / 5)
        n =page_n -page_n % 10
        m= page_n %10+3
    
        # 키워드 수집
        keywords = driver.find_elements_by_class_name('book_keyword > a')
        keyword_pre(keywords)

        # 스코어 수집 
        score = driver.find_element_by_class_name('score').text


        i = 0
        
    
        # 다음 페이지 클릭하기 위한 함수
        while i < page_n :
            # 유저아이디 수집
            ids = driver.find_elements_by_css_selector('.box_detail_review .board_list .comment_wrap .id')
            id_pre(ids)
            # 클로버(평점)수집
            klovers = driver.find_elements_by_css_selector('.comment_wrap .kloverRating > span')
            klovers_pre(klovers)
            # 유저리뷰 수집
            comments = driver.find_elements_by_css_selector('.box_detail_review .board_list .comment_wrap .comment')
            comments_pre(comments)
    
            # 다음페이지 클릭
            i +=1
            time.sleep(1) 
    
            if i> n or k_total <= 50:
                index = m
            else:
                index = 13 
    
            if i < page_n:
        
                css_selector= '.box_detail_review > div.list_paging.align_center > div > a:nth-child({})'.format(index)
                driver.find_element_by_css_selector(css_selector).click()
            else:
                print(j,'th done:',title)
                driver.close()

            time.sleep(2)
            
            data = [(id_list,klover_list,comment_list)]
            lable = ['id','klover','comment']
            df_temp= pd.DataFrame(zip(id_list,klover_list,comment_list),columns=lable)
            # 키워드, 스코어, 리뷰 수 컬럼 추가
            keyword_col = ' '.join(keyword_list)
            df_temp['keyword'] = keyword_col
            df_temp['score']= score
            df_temp['total'] = k_total
            df_temp['title'] = title
            # 도서명으로 신규파일 저장
            df_temp.to_excel('{}.xlsx'.format(title))
            
    except:
            driver.close()
            print(j,'th error:',title) # 에러 발생할 경우 해당 서적 출력
    
    df_keyword = df_keyword.append([df_temp.iloc[0]])
    df_keyword.to_excel

In [5]:
data = pd.DataFrame( columns=['id', 'klover','comment','keyword','score','total','title'])
for i in range(0, len(df)):
    try:
        title = df['도서명'][i]
        df1 = pd.read_excel('{}.xlsx'.format(title))
        data = pd.concat([data, df1])
    except:
        pass